#  0. Preliminaries / Объявление библиотек

In [1]:
# ML Lybriaryis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn import model_selection               # choise model for training
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score        #accuracy of testing


import numpy as np
import pandas as pd
import random

#параметр для буфера вывода. Не режет OUT массив при визуализации
np.set_printoptions(threshold=np.nan)

# 1.  Preprocessing / Предпроцессинг

### LOAD NEGATIVE/POSITIVE DATASET

In [2]:
#переменная с отзывами
data = [] 
#переменная с признаками
data_labels = []

# Загрузка очищенного dataset c позитивными отзывами     
with open("C:/Python/Python36-32/Scripts/Chatting/datasets/data/5_dataset_pos.csv", 'r', encoding='utf-8') as f:
    for i in f:
        data.append(i)
        data_labels.append('POS')              
        
# Загрузка очищенного dataset c негативными отзывами
with open("C:/Python/Python36-32/Scripts/Chatting/datasets/data/5_dataset_neg.csv", 'r', encoding='utf-8') as f:
    for i in f:
        data.append(i)
        data_labels.append('NEG')   

# Загрузка очищенного dataset c негативными отзывами
with open("C:/Python/Python36-32/Scripts/Chatting/datasets/data/5_dataset_netral.csv", 'r', encoding='utf-8') as f:
    for i in f:
        data.append(i)
        data_labels.append('NETRAL')  
    
# обрезаем  окончание  /n
data =  list(map(lambda i: i.strip(), data))
data =  list(map(lambda i: i.replace('\ufeff',''), data))

# 2. Extracting features from text files

## Bags of words

In [3]:
## токенизация
## Преобразование слов в векторы признаков
## v1
my_vectorizer = CountVectorizer(
    analyzer = 'char_wb',
    lowercase = True
    ,ngram_range=(3,3)
    #монограммы и биграммы  
)

## Вызываем метод fit_transform
my_bag = my_vectorizer.fit_transform(data)
my_bag.shape

(1570, 4163)

In [5]:
## v2
my_vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = True
    ,ngram_range=(1,3)
    #монограммы и биграммы  
)

## Вызываем метод fit_transform
my_bag = my_vectorizer.fit_transform(data)
my_bag.shape

(1570, 23578)

### From occurrences to frequencies  / От колличества вхождений к частоте

In [4]:
tfidf_transformer = TfidfTransformer()
my_bag_tfidf = tfidf_transformer.fit_transform(my_bag)
print(my_bag_tfidf.shape)

my_bag_nd = my_bag_tfidf.toarray()

(1570, 4163)


# Training a classifier

In [5]:
# параметры для обучения
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(
        my_bag_nd,
        data_labels,
        test_size=0.60, 
        train_size=0.40,
        random_state=1234
)

#u dont specify the random_state in your code, then every time you run(execute) your code a new random value
#is generated and the train and test datasets would have different values each time.

========================================================================================================================

In [6]:
# Test options and evaluation metric
scoring = 'accuracy'
seed = 7

In [7]:
models = []
models.append(('LinearSVC', LinearSVC())) ###### !!!!!!!!!!!!!!!
models.append(('SVM', SVC(kernel='linear', C=1))) ###### !!!!!!!!!!!!!!!
models.append(('LR', LogisticRegression()))
#models.append(('NBSVM', NBSVM()))

In [8]:
%%time
# evaluate each model in turn
results = []
names = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=5, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    print("Accuracy %s: %0.2f (+/- %0.2f)" % (name, cv_results.mean(), cv_results.std() * 2))

Accuracy LinearSVC: 0.89 (+/- 0.03)
Accuracy SVM: 0.89 (+/- 0.05)
Accuracy LR: 0.86 (+/- 0.06)
Wall time: 10.1 s


In [9]:
models = []
models.append(('CART', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('RFC', RandomForestClassifier(max_depth=2, random_state=0)))

In [10]:
%%time
# evaluate each model in turn
results = []
names = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=5, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    #msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    #print(msg)
    print("Accuracy %s: %0.2f (+/- %0.2f)" % (name, cv_results.mean(), cv_results.std() * 2))

Accuracy CART: 0.83 (+/- 0.06)
Accuracy KNN: 0.84 (+/- 0.04)
Accuracy RFC: 0.74 (+/- 0.12)
Wall time: 8.42 s


# Model predictions

In [11]:
from sklearn.svm import SVC
clfSVM_line = SVC(kernel='linear', C=1).fit(X_train, y_train)

In [12]:
print('SVM :' +  str(clfSVM_line.score(X_test, y_test)))

SVM :0.8842887473460722


In [13]:
y_pred_clfSVM_line = clfSVM_line.predict(X_test)

In [14]:
# тоже что показывает clfNB.score(X_test, y_test) 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [15]:
print('-clfSVM_line----\n')
print(accuracy_score(y_test, y_pred_clfSVM_line))

print('-----\n')
print(confusion_matrix(y_test, y_pred_clfSVM_line))

print('-----\n')
print(classification_report(y_test, y_pred_clfSVM_line))

-clfSVM_line----

0.8842887473460722
-----

[[276  51   0]
 [ 54 237   4]
 [  0   0 320]]
-----

             precision    recall  f1-score   support

        NEG       0.84      0.84      0.84       327
     NETRAL       0.82      0.80      0.81       295
        POS       0.99      1.00      0.99       320

avg / total       0.88      0.88      0.88       942




=============================================== DecisionTreeClassifier =========================================================

In [16]:
clfCART = DecisionTreeClassifier().fit(X_train, y_train)
print('DecisionTreeClassifier :' + str(clfCART.score(X_test, y_test))) 


y_pred_clfCART = clfCART.predict(X_test)

print('-clfCART----\n')
print(accuracy_score(y_test, y_pred_clfCART))
print('-----\n')
print(confusion_matrix(y_test, y_pred_clfCART))
print('-----\n')
print(classification_report(y_test, y_pred_clfCART))
print('-----\n')
print(classification_report(y_test, y_pred_clfCART))

DecisionTreeClassifier :0.8609341825902336
-clfCART----

0.8609341825902336
-----

[[263  58   6]
 [ 63 228   4]
 [  0   0 320]]
-----

             precision    recall  f1-score   support

        NEG       0.81      0.80      0.81       327
     NETRAL       0.80      0.77      0.78       295
        POS       0.97      1.00      0.98       320

avg / total       0.86      0.86      0.86       942

-----

             precision    recall  f1-score   support

        NEG       0.81      0.80      0.81       327
     NETRAL       0.80      0.77      0.78       295
        POS       0.97      1.00      0.98       320

avg / total       0.86      0.86      0.86       942



In [18]:
# тоже что показывает clfNB.score(X_test, y_test) 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred_clfCART))
print(classification_report(y_test, y_pred_clfCART))

0.8609341825902336
             precision    recall  f1-score   support

        NEG       0.81      0.80      0.81       327
     NETRAL       0.80      0.77      0.78       295
        POS       0.97      1.00      0.98       320

avg / total       0.86      0.86      0.86       942



### REVIEW RESULT OF PREDICTIONS

In [21]:
for i in range(0,len(X_test)):
    #print(str(i) + '_______')
    #print('Predicted: ' + y_pred[i]) # pos / neg уже предсказанное у него идекс тот же что и у 
    ind = my_bag_nd.tolist().index(X_test[i].tolist())
    if y_pred_clfSVM_line[i] != data_labels[ind]:
        print(str(i) + '_______')
        print('Predicted: ' + y_pred_clfSVM_line[i]) # pos / neg уже предсказанное у него идекс тот же что и у 
        print('Real: ' + data_labels[ind] + ' - ' + data[ind].strip())
        print('----=========================  DON`T PREDICTED !!! =======================---\n'  + '\n')

6_______
Predicted: NEG
Real: NETRAL - здравствуйте продлила срок задержки заказа отделении пришло сообщение заказ нужно забрать завтрашнего
----=========================  DON`T PREDICTED !!! =======================---


9_______
Predicted: NEG
Real: NETRAL - подали уточнить информацию возврату понедельник получим ответ проинформируем письмом
----=========================  DON`T PREDICTED !!! =======================---


21_______
Predicted: NEG
Real: NETRAL - такая произведен возрат товара личном была подвязана неактуальная
----=========================  DON`T PREDICTED !!! =======================---


22_______
Predicted: NEG
Real: NETRAL - очень поступаете своими постоянными клиентами
----=========================  DON`T PREDICTED !!! =======================---


23_______
Predicted: NETRAL
Real: NEG - возмущена свои ошибки гривен кладете счет ужас
----=========================  DON`T PREDICTED !!! =======================---


45_______
Predicted: NEG
Real: NETRAL - добрий проблему 

353_______
Predicted: NEG
Real: NETRAL - самым теряете свои
----=========================  DON`T PREDICTED !!! =======================---


357_______
Predicted: NETRAL
Real: NEG - двое суток прошло момента повторной вопрос решен
----=========================  DON`T PREDICTED !!! =======================---


365_______
Predicted: NETRAL
Real: NEG - ныне давайте отменять возвращать бред
----=========================  DON`T PREDICTED !!! =======================---


391_______
Predicted: NEG
Real: NETRAL - меня никакого стандартный возврат хочу получить свои деньги
----=========================  DON`T PREDICTED !!! =======================---


398_______
Predicted: NEG
Real: NETRAL - добрий проблему поверненням коштів заказу прийняли товар замовленню кошти повернули тільки
----=========================  DON`T PREDICTED !!! =======================---


406_______
Predicted: NEG
Real: NETRAL - меня вопрос касательно были приобретена кроссовки пары мужские пума найк общей стоимостью около
-

791_______
Predicted: NEG
Real: NETRAL - заказ произошло возврата денег персональный счёт после отмены товара заказ
----=========================  DON`T PREDICTED !!! =======================---


794_______
Predicted: NETRAL
Real: NEG - возврат карту прошел менеджер банка карте такого возврата видит
----=========================  DON`T PREDICTED !!! =======================---


797_______
Predicted: NETRAL
Real: NEG - получу заказ выставленные точные
----=========================  DON`T PREDICTED !!! =======================---


812_______
Predicted: NETRAL
Real: NEG - добрый сделала возврат товара пункте самовывоз академика номер заказа деньги личном кабинете информации товар принят пункте самовывоза адресу павлова сделали отправили возврат киев проблемы киевеуточните информацию возврату
----=========================  DON`T PREDICTED !!! =======================---


815_______
Predicted: POS
Real: NETRAL - спасибо очень
----=========================  DON`T PREDICTED !!! ==============



# Save model

In [ ]:
import pickle
s = pickle.dumps(clfCART)
clfCART2 = pickle.loads(s)
# используем обученный алгоритм на повторной выборке
# log_model2.predict(X[0:1])

In [ ]:
from sklearn.externals import joblib
# сохранили файли конфиг
joblib.dump(clfCART, 'my_save_algo.pkl')